In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import numpy as np

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])
def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [3]:
use_cuda = not False and torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
file_name = 'interpolated_adversarial_training'
net_1 = ResNet18()
net_1 = net_1.to(device)
cudnn.benchmark = True
checkpoint = torch.load('./checkpoint/' + file_name)
net_1.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [7]:
file_name = 'pgd_adversarial_training'
net_2 = ResNet18()
net_2 = net_2.to(device)
cudnn.benchmark = True
checkpoint = torch.load('./checkpoint/' + file_name)
net_2.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [21]:
# Combine the two models into an ensemble
class Ensemble(nn.Module):
    def __init__(self, model1, model2):
        super(Ensemble, self).__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, x):
        output1 = self.model1(x)
        output2 = self.model2(x)
        return (output1 + output2) / 2

ensemble_model = Ensemble(net_1, net_2)

In [22]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
l = [x for (x, y) in test_loader]
x_test = torch.cat(l, 0)
l = [y for (x, y) in test_loader]
y_test = torch.cat(l, 0)
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy
eval_test(ensemble_model, device, test_loader)

Files already downloaded and verified
Test: Average loss: 0.4227, Accuracy: 8834/10000 (88%)


(0.4227170991897583, 0.8834)

In [23]:
!pip install git+https://github.com/fra31/auto-attack

  Cloning https://github.com/fra31/auto-attack to c:\users\zahra\appdata\local\temp\pip-req-build-ersyaoh3
  Resolved https://github.com/fra31/auto-attack to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack 'C:\Users\zahra\AppData\Local\Temp\pip-req-build-ersyaoh3'


In [24]:
from autoattack import AutoAttack
adversary = AutoAttack(ensemble_model, norm='Linf', eps=0.031, version='standard')

setting parameters for standard version


In [25]:
adv_complete = adversary.run_standard_evaluation(x_test, y_test,bs=100)

using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 88.34%
apgd-ce - 1/89 - 48 out of 100 successfully perturbed
apgd-ce - 2/89 - 47 out of 100 successfully perturbed
apgd-ce - 3/89 - 48 out of 100 successfully perturbed
apgd-ce - 4/89 - 47 out of 100 successfully perturbed
apgd-ce - 5/89 - 43 out of 100 successfully perturbed
apgd-ce - 6/89 - 45 out of 100 successfully perturbed
apgd-ce - 7/89 - 50 out of 100 successfully perturbed
apgd-ce - 8/89 - 42 out of 100 successfully perturbed
apgd-ce - 9/89 - 44 out of 100 successfully perturbed
apgd-ce - 10/89 - 43 out of 100 successfully perturbed
apgd-ce - 11/89 - 44 out of 100 successfully perturbed
apgd-ce - 12/89 - 45 out of 100 successfully perturbed
apgd-ce - 13/89 - 39 out of 100 successfully perturbed
apgd-ce - 14/89 - 46 out of 100 successfully perturbed
apgd-ce - 15/89 - 47 out of 100 successfully perturbed
apgd-ce - 16/89 - 42 out of 100 successfully perturbed
apgd-ce - 17/89 - 43 out of 100 success

In [27]:
adv_loader_Linf = torch.utils.data.DataLoader(adv_complete, batch_size=100, shuffle=False, num_workers=2)
num_total_images = 0
num_successful_attacks = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = ensemble_model(images)
        _, predicted = torch.max(outputs.data, 1)
    for adv_images in adv_loader_Linf:
        adv_images = adv_images.to(device)
        outputs_adv = ensemble_model(adv_images)
        _, predicted_adv = torch.max(outputs_adv.data, 1)

        # Count the number of adversarial examples that were successfully attacked
        for i in range(len(images)):
            if predicted[i] != predicted_adv[i]:
                num_successful_attacks += 1
        
        # Increment the total number of images
        num_total_images += len(images)

# Calculate the adversarial attack rate
attack_rate = (num_successful_attacks / num_total_images) * 100
print(f"Adversarial attack rate: {attack_rate:.2f}%")

Adversarial attack rate: 89.41%


In [30]:
from autoattack import AutoAttack
adversary_L2 = AutoAttack(ensemble_model, norm='L2', eps=0.031, version='standard')
adv_complete_L2 = adversary_L2.run_standard_evaluation(x_test, y_test,bs=100)

setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 88.34%
apgd-ce - 1/89 - 0 out of 100 successfully perturbed
apgd-ce - 2/89 - 1 out of 100 successfully perturbed
apgd-ce - 3/89 - 2 out of 100 successfully perturbed
apgd-ce - 4/89 - 3 out of 100 successfully perturbed
apgd-ce - 5/89 - 0 out of 100 successfully perturbed
apgd-ce - 6/89 - 2 out of 100 successfully perturbed
apgd-ce - 7/89 - 1 out of 100 successfully perturbed
apgd-ce - 8/89 - 1 out of 100 successfully perturbed
apgd-ce - 9/89 - 0 out of 100 successfully perturbed
apgd-ce - 10/89 - 2 out of 100 successfully perturbed
apgd-ce - 11/89 - 1 out of 100 successfully perturbed
apgd-ce - 12/89 - 3 out of 100 successfully perturbed
apgd-ce - 13/89 - 0 out of 100 successfully perturbed
apgd-ce - 14/89 - 1 out of 100 successfully perturbed
apgd-ce - 15/89 - 2 out of 100 successfully perturbed
apgd-ce - 16/89 - 1 out of 100 successfully perturbed
apgd-ce - 17/89

KeyboardInterrupt: 

In [ ]:
adv_loader_L2 = torch.utils.data.DataLoader(adv_complete_L2, batch_size=100, shuffle=False, num_workers=2)
num_total_images = 0
num_successful_attacks = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
    for adv_images in adv_loader_L2:
        adv_images = adv_images.to(device)
        outputs_adv = ensemble_model(adv_images)
        _, predicted_adv = torch.max(outputs_adv.data, 1)

        # Count the number of adversarial examples that were successfully attacked
        for i in range(len(images)):
            if predicted[i] != predicted_adv[i]:
                num_successful_attacks += 1
        
        # Increment the total number of images
        num_total_images += len(images)

# Calculate the adversarial attack rate
attack_rate = (num_successful_attacks / num_total_images) * 100
print(f"Adversarial attack rate: {attack_rate:.2f}%")